In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import csv
import sys,os
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


# for named entity
from nltk import word_tokenize, pos_tag, ne_chunk, tree, download
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# for preparing data
download('vader_lexicon')
%matplotlib inline

#!export GOOGLE_APPLICATION_CREDENTIALS='sentimental.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='sentimental.json'

# nltk for natural language processing

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/chitrankdixit/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
path =r'./playstore_corpus_tsv/' # use your path
allFiles = glob.glob(path + "/*.tsv")

# for out_count, file in enumerate(allFiles):
#     with open(file, newline='') as csvfile:
#         spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#         #import pdb;pdb.set_trace()
#         for counter, row in enumerate(spamreader):
#             import pdb;pdb.set_trace()
#             print(row)
            
#     with open(file+out_count, 'wt') as csvfile:
#             writer = csv.writer(csvfile, delimiter='', quotechar='|')
#             #writer.writerow(["#"] + anarkali_characteristics)
#             #import pdb;pdb.set_trace()
#             for element in lehenga_characteristics:
#                 #import pdb;pdb.set_trace()
#                 writer.writerow([element["name"], element["count"]])

In [15]:
# get full data


frame = pd.DataFrame()
list_ = []

# datacolumns = [
#     "Package Name",
#     "App Version Code",
#     "Reviewer Language",
#     "Device",
#     "Review Submit Date and Time",
#     "Review Submit Millis Since Epoch",
#     "Review Last Update Date and Time",
#     "Review Last Update Millis Since Epoch",
#     "Star Rating",
#     "Review Title",
#     "Review Text",
#     "Developer Reply Date and Time",
#     "Developer Reply Millis Since Epoch",
#     "Developer Reply Text,Review Link"
# ]

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')
    list_.append(df)
    
    
full_data = pd.concat(list_)
full_data.head(5)

#mail_data = pd.read_csv("parsed_data.tsv",index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')

,Description,Subject,sentiment,tag
0,This is a brilliant concept. Please start for ...,Great product!,2,"["" Please start for men's asap""]"
1,Brilliant concept :),NaN,2,['Brilliant concept :)']
2,NaN,NaN,-1,[]
3,Brilliant stuff!! Waiting to see more..,NaN,1,['']
4,NaN,NaN,-1,[]


In [16]:
import pdb;pdb.set_trace()
dir(full_data)

--Return--
> <ipython-input-16-e08393cc6d35>(1)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) full_data
                                           Description  \
0    This is a brilliant concept. Please start for ...   
1                                 Brilliant concept :)   
2                                                  NaN   
3              Brilliant stuff!! Waiting to see more..   
4                                                  NaN   
5    This idea will be considered as one of the top...   
6                                                  NaN   
7    No spending on buying apparels for gf... Why t...   
8    Brilliant idea!! Adorably splendid dresses at ...   
9                                                  NaN   
10                                                 NaN   
11                                                 NaN   
12                                                 NaN   
13   The whole concept looks great. Something to re...   
14                  

(Pdb) full_data['sentiment'][0]
0    2
0    2
0    2
0    2
0    2
0    2
0    2
0    2
0    2
0    2
0    2
Name: sentiment, dtype: int64
(Pdb) c


['Description',
 'Subject',
 'T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '_

In [17]:
# remove the NaN 
full_data["Subject"] = full_data["Subject"].fillna('')
full_data["Description"] = full_data["Description"].fillna('')
full_data["Sentiment"] = full_data["sentiment"].fillna(0)
#full_data["tag"] = full_data["tag"].fillna('')
# full_data["Developer Reply Date and Time"] = full_data["Developer Reply Date and Time"].fillna('')
# full_data["Developer Reply Millis Since Epoch"] = full_data["Developer Reply Millis Since Epoch"].fillna('')
# full_data["Developer Reply Text"] = full_data["Developer Reply Text"].fillna('')

# slicing the required columns only
full_data = full_data[["Subject", "Description", "sentiment", "tag"]]

In [18]:
# clean mail data
#mail_data

In [26]:
full_data.loc[full_data['sentiment'] == 0].__len__()

482

In [19]:
# full_data.columns = ["Description","Subject"]
full_data.head()

,Subject,Description,sentiment,tag
0,Great product!,This is a brilliant concept. Please start for ...,2,"["" Please start for men's asap""]"
1,,Brilliant concept :),2,['Brilliant concept :)']
2,,,-1,[]
3,,Brilliant stuff!! Waiting to see more..,1,['']
4,,,-1,[]


In [20]:
# mail_data = mail_data[["subject", "description"]]
# mail_data.columns = ["Subject", "Description"]
# mail_data.head()

# when mail data get cleaned properly then use the below
#full_data = full_data.append(mail_data)

In [21]:
full_data.tail()

,Subject,Description,sentiment,tag
995,,Wow! Never knew that it might be this smooth t...,2,['']
996,,A huge thank you for making it so easy and fun...,2,['']
997,,It is very good app,2,['It is very good app']
998,,It is so higher to lease cloths! Facilitates m...,1,[' So higher and high-quality']
999,,It's miles higher manner to lease cloths. I am...,2,"[""It's miles higher manner to lease cloths""]"


In [24]:
full_data = full_data.loc[full_data['Description'] != '']


full_data.head()

,Subject,Description,sentiment,tag
0,Great product!,This is a brilliant concept. Please start for ...,2,"["" Please start for men's asap""]"
1,,Brilliant concept :),2,['Brilliant concept :)']
3,,Brilliant stuff!! Waiting to see more..,1,['']
5,Excellent,This idea will be considered as one of the top...,2,['']
7,Brilliant ..this is what we needed!!,No spending on buying apparels for gf... Why t...,1,[' Why to buy if u can rent it']


In [25]:
len(full_data)

8669

In [38]:
sid = SentimentIntensityAnalyzer()
client = language.LanguageServiceClient()

# add a named entities to look for
#import pdb;pdb.set_trace()
negation_list = ["No", "Not"]

tech_list = ["order", "placed", "tech", "issue", "coupon", "item", "order placed", "coupon issue", "tech issue", "App crash", "web crash", "ios", 
            "android", "signup issue", "signin issue", "order placed", "order confirmation", "online payment", "hang", "too slow"
            , "cart", "image not matching"]
ops_list = ["delivery", "measurement", "pickup", "refund",  "fitting", "trial", "pickup", "refund", "deposit", "on-time", "delay", "address updation", "return", "cancellation", "exchange",
"payment issues"]
product_list = ["content", "available", "error", "loading", "website not loading", "site not loading", "content not available", "cloth tear", "defective product", "discount codes"]

def get_sentiment_score(item):
    sentences = item['Description'].split('.')#.encode('utf-16')
    #import pdb;pdb.set_trace()
    #a = b
    sentiment = None
    sentence_dict = {}
    min_score = 100
    min_text_tag_list = []
    for text in sentences:
        document = types.Document(
            content=text,
            type=enums.Document.Type.PLAIN_TEXT)

        # Detects the sentiment of the text
        
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        if sentiment.score < min_score:
            min_score = sentiment.score
            min_text_tag_list.append(text) 
        sentence_dict.update({text: sentiment.score})
        
    sentiment_score= sum(sentence_dict.values()) / len(sentence_dict.values())
    if sentiment_score <= -0.25 and sentiment_score > -1.0:
        sentiment = 0
    elif sentiment_score <= 0.25 and sentiment_score > -0.25:
        sentiment = 1
    elif sentiment_score <= 1.00 and sentiment_score > 0.25:
        sentiment = 2
        
    
    
    
    
    

#     negate = None
#     for element in negation_list:
#         m = re.findall('^'+element+' \w+', sentences)
#         if m:
#             break
#     if m:
#         negate, sentences = sentences[:2],sentences[2:]
#     ss = sid.polarity_scores(sentences)
#     if negate:
#         if ss['neg'] > max(ss['pos'], ss['neu']): 
#             ss['pos'] = 1.0
#             ss['neu'] = 0.0
#             ss['neg'] = 0.0
#         elif ss['pos'] > max(ss['neg'], ss['neu']): 
#             ss['pos'] = 0.0
#             ss['neu'] = 0.0
#             ss['neg'] = 1.0
# #         elif ss['neu'] > max(ss['pos'], ss['neg']):
# #             ss['pos'] = 1.0
# #             ss['neu'] = 0.0
# #             ss['neg'] = 0.0
    
#     negative = ss["neg"]
#     positive = ss["pos"]
#     neutral = ss["neu"]
    #compound = ss["compound"]
    return pd.Series({'sentiment': sentiment, "tag": min_text_tag_list})


scores = full_data.apply(get_sentiment_score, axis=1)

final_data = pd.concat([ full_data , scores], axis=1)

KeyboardInterrupt: 

In [114]:
# def get_sentiment_class(item):
#     sentiment_class = 0
#     score_list = [item['neutral'], item['negative'], item['positive']]
#     max_val = max(score_list)
    
#     sentiment_class = score_list.index(max_val)
    
#     return pd.Series({'sentiment_class': sentiment_class})


# score_class = final_data.apply(get_sentiment_class, axis=1)

# final_sliced_data = pd.concat([ final_data , score_class], axis=1)


In [115]:
# bring the data with probability tech + , tech - , ops + , ops - , product + , product -
import re
tech_list = ["order", "placed", "tech", "issue", "coupon", "item", "order placed", "coupon issue", "tech issue", "App crash", "web crash", "ios", 
            "android", "signup issue", "signin issue", "order placed", "order confirmation", "online payment", "hang", "too slow"
            , "cart", "image not matching"]
ops_list = ["delivery", "measurement", "pickup", "refund",  "fitting", "trial", "pickup", "refund", "deposit", "on-time", "delay", "address updation", "return", "cancellation", "exchange",
"payment issues"]
product_list = ["content", "available", "error", "loading", "website not loading", "site not loading", "content not available", "cloth tear", "defective product", "discount codes"]

def calculate_token_probability(item):
    tech = 0; ops = 0; product = 0
    tokens = item['Description'].split(' ')
#     for element in tokens:
#         #import pdb;pdb.set_trace()
#         try:
#             tech_match = re.findall(element.lower(), tech_str)
#         except Exception:
#             tech_match = None
            
#         try:
#             ops_match = re.findall(element.lower(), ops_str)
#         except Exception:
#             ops_match = None
            
#         try:
#             product_match = re.findall(element.lower(), product_str)
#         except Exception:
#             product_match = None
#         if tech_match:
#             tech += 1
#         elif ops_match:
#             ops += 1
#         elif product_match:
#             product += 1
    tech = [1 for tech_i in tech_list if tech_i.lower() in item['Description']]
    ops = [1 for ops_i in ops_list if ops_i.lower() in item['Description']]
    product = [1 for product_i in product_list if product_i.lower() in item['Description']]
    

#     for tech_i in tech_list:
#         try:
#             tech_match = re.findall(tech_i.lower(), item['Review Text'])
#         except Exception:
#             tech_match = None
            
#         if tech_match:
#              tech += 1
    
#     for ops_i in ops_list:
#         try:
#             ops_match = re.findall(ops_i.lower(), item['Review Text'])
#         except Exception:
#             ops_match = None
            
#         if ops_match:
#              ops += 1
    
#     for product_i in product_list:
#         try:
#             product_match = re.findall(product_i.lower(), item['Review Text'])
#         except Exception:
#             product_match = None
            
#         if product_match:
#              product += 1
            
    tech_sum = sum(tech)
    ops_sum = sum(ops)
    product_sum = sum(product)
    
    sum_all = tech_sum + ops_sum + product_sum
    try:
        return pd.Series({'tech_prob': tech_sum/sum_all, 'ops_prob': ops_sum/sum_all, 'product_prob': product_sum/sum_all})
    except Exception: 
        return pd.Series({'tech_prob': 0, 'ops_prob': 0, 'product_prob': 0})
    
    
# def cal(item):
#     print(item['Review Text'])
#     return pd.Series({'hello': item}, )
    
# probability = sliced_data.apply(calculate_token_probability, axis=1)

# final_sliced_data = pd.concat([ sliced_data , probability], axis=1)


# final_sliced_data['ops_positive'] = (( final_sliced_data['ops_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Positive"] == True).astype(int).values
# final_sliced_data['product_positive'] = (( final_sliced_data['product_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Positive"] == True ).astype(int).values
# final_sliced_data['tech_positive'] = (( final_sliced_data['tech_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1)  ) & final_sliced_data["Positive"] == True).astype(int).values
# final_sliced_data['ops_negative'] = (( final_sliced_data['ops_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data['product_negative'] = (( final_sliced_data['product_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data['tech_negative'] = (( final_sliced_data['tech_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data

In [116]:
final_sliced_data['Description'].head(5)

0    This is a brilliant concept. Please start for ...
1                                 Brilliant concept :)
3              Brilliant stuff!! Waiting to see more..
5    This idea will be considered as one of the top...
7    No spending on buying apparels for gf... Why t...
Name: Description, dtype: object

In [117]:
final_sliced_data.head(5)

,Description,Subject,negative,neutral,positive,sentiment_class
0,This is a brilliant concept. Please start for ...,Great product!,0.0,0.534,0.466,0
1,Brilliant concept :),,0.0,0.128,0.872,2
3,Brilliant stuff!! Waiting to see more..,,0.0,0.533,0.467,0
5,This idea will be considered as one of the top...,Excellent,0.0,0.715,0.285,0
7,No spending on buying apparels for gf... Why t...,Brilliant ..this is what we needed!!,0.0,1.000,0.000,0


In [118]:
# generate csv file
final_sliced_data.to_csv("text_data.tsv", sep='\t')

In [119]:
final_sliced_data.to_csv('pos_neg_score.csv', index = False)

In [120]:
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 0].__len__())
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 1].__len__())
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 2].__len__())

7999
30
640


In [121]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy


def format_sentence(sent):
    return({word: True for word in word_tokenize(sent)})

In [122]:
labels = final_sliced_data["sentiment_class"].values;

#import pdb;pdb.set_trace()  
kf = StratifiedKFold(n_splits=4)


totalNBV = 0

totalMatNBV = np.zeros((3,3));

# tr, te = kf.split(sliced_data['Review Text'], labels)
# td = []
# for i in train_index:
#     td.append(sliced_data['Review Text'][i])
#import pdb;pdb.set_trace()
review_text = final_sliced_data['Description'].tolist()

# break words
review_words = []

# for item in review_text:
#     review_words += item.split(" ")
counter = 0
for train_index, test_index in kf.split(review_text, labels):
    #import pdb;pdb.set_trace()
    #print(train_index.__len__(), test_index.__len__())
    # X_train = [review_text[i] for i in train_index]
    # X_test = [review_text[i] for i in test_index]
    #import pdb;pdb.set_trace()
    X_train = [[format_sentence(review_text[i]), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in train_index]
    X_test = [[format_sentence(review_text[i]), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in test_index]
    
    X_train_nb = [[list(map(int, format_sentence(review_text[i]).values())), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in train_index]
    X_test_nb = [[list(map(int, format_sentence(review_text[i]).values())), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    #vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=False, stop_words='english')
    #train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
    #test_corpus_tf_idf = vectorizer.transform(X_test)
    nb = MultinomialNB()#, GaussianNB
    #import pdb;pdb.set_trace()
    nb.fit(X_train_nb, y_train)
    nb.predict(X_test_nb)
    nbc = NaiveBayesClassifier.train(X_train)
    print(accuracy(nbc, X_test))
    print(accuracy(nb, X_test_nb))
    
    example1 = "Cats are awesome!"
 
    print(nbc.classify(format_sentence(example1)))
    
    example2 = "I don’t like cats."
 
    print(nbc.classify(format_sentence(example2)))
    result1 = []
    for item in X_test:
        result1.append(nbc.classify(item[0]))

ValueError: setting an array element with a sequence.

In [ ]:
ss = sid.polarity_scores(sentences)